# Анализ результатов A/B-тестирования

Представьте, что к вам обратились представители интернет-магазина BitMotion Kit, в котором продаются геймифицированные товары для тех, кто ведёт здоровый образ жизни. У него есть своя целевая аудитория, даже появились хиты продаж: эспандер со счётчиком и напоминанием, так и подстольный велотренажёр с Bluetooth.

В будущем компания хочет расширить ассортимент товаров. Но перед этим нужно решить одну проблему. Интерфейс онлайн-магазина слишком сложен для пользователей — об этом говорят отзывы.

Чтобы привлечь новых клиентов и увеличить число продаж, владельцы магазина разработали новую версию сайта и протестировали его на части пользователей. По задумке, это решение доказуемо повысит количество пользователей, которые совершат покупку.

Ваша задача — провести оценку результатов A/B-теста. В вашем распоряжении:

* данные о действиях пользователей и распределении их на группы,

* техническое задание.

Оцените корректность проведения теста и проанализируйте его результаты.

## 1. Опишите цели исследования.


Целью проекта является оценка данных A/B-теста, представленных интернет-магазином BitMotion Kit. 

Для проведения оценки необходимо:
- оценить достаточность и адекватность данных,
- провести очистку данных от дубликатов,
- выделить пользователей, участвующих в тесте,
- проверить соответствие требованиям технического задания,
- проверить равномерность распределения пользователей по группам теста,
- проверить отсутствие пересечений с конкурирующим тестом,
- определить горизонт анализа,
- оценить достаточность выборки для получения статистически значимых результатов,
- рассчитать для каждой группы количество посетителей, сделавших покупку, и общее количество посетителей, и конверсию,
- оценить изменение конверсии подходящим статистическим тестом, учитывая все этапы проверки гипотез,
- описать выводы по проведённой оценке результатов A/B-тестирования.

## 2. Загрузите данные, оцените их целостность.


In [23]:
# Импортируем библиотеки
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import numpy as np
from scipy.stats import f_oneway
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportions_ztest


In [3]:
participants = pd.read_csv('https://code.s3.yandex.net/datasets/ab_test_participants.csv')
events = pd.read_csv('https://code.s3.yandex.net/datasets/ab_test_events.zip',
                     parse_dates=['event_dt'], low_memory=False)

In [4]:
participants.info()

participants.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
 3   device   14525 non-null  object
dtypes: object(4)
memory usage: 454.0+ KB


,user_id,group,ab_test,device
0,0002CE61FF2C4011,B,interface_eu_test,Mac
1,001064FEAAB631A1,B,recommender_system_test,Android
2,001064FEAAB631A1,A,interface_eu_test,Android
3,0010A1C096941592,A,recommender_system_test,Android
4,001E72F50D1C48FA,A,interface_eu_test,Mac
5,002412F1EB3F6E38,B,interface_eu_test,Mac
6,002540BE89C930FB,B,interface_eu_test,Android
7,0031F1B5E9FBF708,A,interface_eu_test,Android
8,003346BB64227D0C,B,interface_eu_test,Android
9,00341D8401F0F665,A,recommender_system_test,iPhone


Данные анализа достаточны и целостностны. Пропуски отсутствуют. Проверим наличие дубликатов:

In [5]:
duplicates_mask = participants.duplicated()
duplicated_rows = participants[duplicates_mask]
print(duplicated_rows)

Empty DataFrame
Columns: [user_id, group, ab_test, device]
Index: []


Дубликаты отсутствуют.

In [6]:
events.info()

events.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787286 entries, 0 to 787285
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     787286 non-null  object        
 1   event_dt    787286 non-null  datetime64[ns]
 2   event_name  787286 non-null  object        
 3   details     249022 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 24.0+ MB


,user_id,event_dt,event_name,details
0,GLOBAL,2020-12-01 00:00:00,End of Black Friday Ads Campaign,ZONE_CODE15
1,CCBE9E7E99F94A08,2020-12-01 00:00:11,registration,0.0
2,GLOBAL,2020-12-01 00:00:25,product_page,NaN
3,CCBE9E7E99F94A08,2020-12-01 00:00:33,login,NaN
4,CCBE9E7E99F94A08,2020-12-01 00:00:52,product_page,NaN
5,AA346F4D22148024,2020-12-01 00:01:46,registration,-2.0
6,7EF01D0E72AF449D,2020-12-01 00:02:06,registration,-5.0
7,9A6276AD14B14252,2020-12-01 00:02:20,registration,-2.0
8,9B186A3B1A995D36,2020-12-01 00:02:37,registration,-3.5
9,9A6276AD14B14252,2020-12-01 00:02:53,login,NaN


 Данные анализа достаточны и целостностны. Данные отсутвуют только в столбце details, что является допустимым. Проверим наличие дубликатов.

In [7]:
# Находим дубликаты
duplicates_mask = events.duplicated()
duplicated_rows = events[duplicates_mask]

# Выводим количество дубликатов
num_duplicates = events.duplicated().sum()

# Выводим сами повторяющиеся строки
print("\nСами повторяющиеся строки:")
print(duplicated_rows)

# Рассчитаем общее количество строк
total_rows = len(events)

# Рассчитаем процент дубликатов
percentage_duplicates = (num_duplicates / total_rows) * 100
print(f"\nОбщее количество строк: {total_rows}")
print(f"Количество дубликатов (вторые и последующие вхождения): {num_duplicates}")
print(f"Процент дубликатов: {percentage_duplicates:.2f}%")


Сами повторяющиеся строки:
                 user_id            event_dt    event_name details
50      A39D63750BBE9B34 2020-12-01 00:08:24         login     NaN
132     631020621D23464A 2020-12-01 00:25:03         login     NaN
278     AD6541E75198ABEF 2020-12-01 00:48:46         login     NaN
390     ADBBC43BED1249C8 2020-12-01 01:06:02  product_cart     NaN
446     928AD890A8E7BDE7 2020-12-01 01:11:10         login     NaN
...                  ...                 ...           ...     ...
787141            GLOBAL 2020-12-31 23:21:18      purchase    4.49
787200            GLOBAL 2020-12-31 23:36:55      purchase    4.49
787209            GLOBAL 2020-12-31 23:39:41  product_cart     NaN
787217  F9C2F1ECC9624248 2020-12-31 23:42:17  product_page     NaN
787263            GLOBAL 2020-12-31 23:53:42      purchase    4.49

[36318 rows x 4 columns]

Общее количество строк: 787286
Количество дубликатов (вторые и последующие вхождения): 36318
Процент дубликатов: 4.61%


Менее 5% данных являются полными дубликатами. Удалим их из анализа.

In [8]:
events_no_duplicates = events.drop_duplicates()

## 3. По таблице `ab_test_participants` оцените корректность проведения теста:

   3\.1 Выделите пользователей, участвующих в тесте, и проверьте:

   - соответствие требованиям технического задания,

   - равномерность распределения пользователей по группам теста,

   - отсутствие пересечений с конкурирующим тестом (нет пользователей, участвующих одновременно в двух тестовых группах).

Мы проверяем следующую гипотезу: упрощение интерфейса приведёт к тому, что в течение семи дней после регистрации в системе конверсия зарегистрированных пользователей в покупателей увеличится как минимум на три процентных пункта.

In [9]:
participants.head()

,user_id,group,ab_test,device
0,0002CE61FF2C4011,B,interface_eu_test,Mac
1,001064FEAAB631A1,B,recommender_system_test,Android
2,001064FEAAB631A1,A,interface_eu_test,Android
3,0010A1C096941592,A,recommender_system_test,Android
4,001E72F50D1C48FA,A,interface_eu_test,Mac


In [10]:
# Оценим распределения пользователей по группам 
A = participants[(participants['ab_test'] == 'interface_eu_test') & (participants['group'] == 'A')]['user_id']

B = participants[(participants['ab_test'] == 'interface_eu_test') & (participants['group'] == 'B')]['user_id']

intersection = list(set(A) & set(B))
print(f"Размер пересечения: {len(intersection)}")
print(f"Пользователи в пересечении: {intersection}")

Размер пересечения: 0
Пользователи в пересечении: []


Пересечения пользователей в группах нет, группы являются независимыми.

In [11]:
# Оценим отсутствие пересечений с конкурирующим тестом 
eu_test = participants[participants['ab_test'] == 'interface_eu_test']['user_id']

system_test = participants[participants['ab_test'] == 'recommender_system_test']['user_id']

intersection = list(set(eu_test) & set(system_test))
print(f"Размер пересечения: {len(intersection)}")

Размер пересечения: 887


В данных присутствуют пользователи, участвовавшие одновременно в двух тестах. Оценим размер групп в нашем тесте и удалим этих пересекающихся пользователей.

In [12]:
print(f"Размер группы А интересующего теста (до удаления): {len(A)}")
print(f"Размер группы B интересующего теста (до удаления): {len(B)}")

# Удаляем пересекающихся пользователей 
users_to_exclude_set = set(intersection) 
eu_test_cleaned_df = participants[
    (~participants['user_id'].isin(users_to_exclude_set)) & 
    (participants['ab_test'] == 'interface_eu_test')  ]

A_cleaned = eu_test_cleaned_df[eu_test_cleaned_df['group'] == 'A']['user_id']

B_cleaned = eu_test_cleaned_df[eu_test_cleaned_df['group'] == 'B']['user_id']

# Выводим размеров групп после удаления пересечений
print(f"Размер группы А (после удаления пересечений): {len(A_cleaned)}")
print(f"Размер группы B (после удаления пересечений): {len(B_cleaned)}")

Размер группы А интересующего теста (до удаления): 5383
Размер группы B интересующего теста (до удаления): 5467
Размер группы А (после удаления пересечений): 4952
Размер группы B (после удаления пересечений): 5011


Полученные после обработки данные показывают, что группы распределены равномерно.

3\.2 Проанализируйте данные о пользовательской активности по таблице `ab_test_events`:

- оставьте только события, связанные с участвующими в изучаемом тесте пользователями;

In [13]:
# Извлекаем user_id всех пользователей изучаемого теста 
participating_user_ids = eu_test_cleaned_df['user_id'].unique()

# Фильтруем ab_test_events, оставляя только нужных пользователей
events_filtered = events[events['user_id'].isin(participating_user_ids)]

events_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73815 entries, 64672 to 780371
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     73815 non-null  object        
 1   event_dt    73815 non-null  datetime64[ns]
 2   event_name  73815 non-null  object        
 3   details     19450 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 2.8+ MB


- определите горизонт анализа: рассчитайте время (лайфтайм) совершения события пользователем после регистрации и оставьте только те события, которые были выполнены в течение первых семи дней с момента регистрации;

In [14]:
# Выведем все уникальные события для пользователей
events_filtered['event_name'].unique()


array(['registration', 'login', 'product_page', 'purchase',
       'product_cart'], dtype=object)

In [16]:
# Определяем дату регистрации для каждого пользователя 
registration_dates = events_filtered[events_filtered['event_name'] == 'registration'].groupby('user_id').agg(
    registration_dt=('event_dt', 'min') ).reset_index()

# Присоединяем дату регистрации к каждому событию пользователя
events_with_registration_dt = pd.merge(
    events_filtered,
    registration_dates,
    on='user_id',
    how='left'
)

# Рассчитываем лайфтайм в днях
events_with_registration_dt['lifetime_days'] = (
    events_with_registration_dt['event_dt'].dt.date - events_with_registration_dt['registration_dt'].dt.date
).dt.days

# Оставляем только те события, которые были выполнены в течение первых семи дней
events_within_7_days = events_with_registration_dt[events_with_registration_dt['lifetime_days'] <= 7].copy()

events_within_7_days.head()

,user_id,event_dt,event_name,details,registration_dt,lifetime_days
0,5F506CEBEDC05D30,2020-12-06 14:10:01,registration,0.0,2020-12-06 14:10:01,0
1,51278A006E918D97,2020-12-06 14:37:25,registration,-3.8,2020-12-06 14:37:25,0
2,A0C1E8EFAD874D8B,2020-12-06 17:20:22,registration,-3.32,2020-12-06 17:20:22,0
3,275A8D6254ACF530,2020-12-06 19:36:54,registration,-0.48,2020-12-06 19:36:54,0
4,0B704EB2DC7FCA4B,2020-12-06 19:42:20,registration,0.0,2020-12-06 19:42:20,0


Оцените достаточность выборки для получения статистически значимых результатов A/B-теста. Заданные параметры:

- базовый показатель конверсии — 30%,

- мощность теста — 80%,

- достоверность теста — 95%.


In [18]:
# Заданные параметры
p1 = 0.30  # Базовая конверсия
p2 = p1 + 0.03  # Целевая конверсия (p1 + MDE, составляющий три процентных пункта согласно исходной гипотезе)
alpha = 0.05  # Уровень значимости (для 95% достоверности)
power = 0.80  # Мощность теста

# Рассчитываем размер эффекта (Cohen's h)
effect_size = 2 * (np.arcsin(np.sqrt(p2)) - np.arcsin(np.sqrt(p1)))

# Инициализируем класс NormalIndPower
power_analysis = NormalIndPower()

# Рассчитываем размер выборки
sample_size = power_analysis.solve_power(
    effect_size=effect_size,
    alpha=alpha,
    power=power,
    ratio=1,  # Равномерное распределение выборок
)

print(f"\nНеобходимый размер выборки для каждой группы: {int(sample_size)}")
print(f"Общий необходимый размер выборки для теста: {int(sample_size * 2)}")
print(f"\nПредоставленный размер группы А: {len(A_cleaned)}")
print(f"Предоставленный размеразмер группы B: {len(B_cleaned)}")
print(f"\nВывод: полученной выборки достаточно для проведения статистической оценки")


Необходимый размер выборки для каждой группы: 3761
Общий необходимый размер выборки для теста: 7523

Предоставленный размер группы А: 4952
Предоставленный размеразмер группы B: 5011

Вывод: полученной выборки достаточно для проведения статистической оценки


- рассчитайте для каждой группы количество посетителей, сделавших покупку, и общее количество посетителей.

In [19]:
# Соединяем обе таблицы
merged_df = pd.merge(
    eu_test_cleaned_df,
    events_within_7_days,
    on='user_id',
    how='left'
)

merged_df.head()

,user_id,group,ab_test,device,event_dt,event_name,details,registration_dt,lifetime_days
0,0002CE61FF2C4011,B,interface_eu_test,Mac,2020-12-07 04:37:31,registration,-2.38,2020-12-07 04:37:31,0
1,0002CE61FF2C4011,B,interface_eu_test,Mac,2020-12-07 04:37:49,login,NaN,2020-12-07 04:37:31,0
2,0002CE61FF2C4011,B,interface_eu_test,Mac,2020-12-07 04:37:57,login,NaN,2020-12-07 04:37:31,0
3,0002CE61FF2C4011,B,interface_eu_test,Mac,2020-12-07 04:38:54,login,NaN,2020-12-07 04:37:31,0
4,0002CE61FF2C4011,B,interface_eu_test,Mac,2020-12-08 22:15:35,login,NaN,2020-12-07 04:37:31,1


In [20]:
# Общее количество посетителей (уникальных user_id) для каждой группы
total_visitors_per_group = merged_df.groupby('group')['user_id'].nunique()
print("\n--- Общее количество посетителей в каждой группе ---")
print(total_visitors_per_group)

# Количество посетителей, сделавших покупку (уникальных user_id с event_name == 'purchase')
purchases_df = merged_df[merged_df['event_name'] == 'purchase']

purchasers_per_group = purchases_df.groupby('group')['user_id'].nunique()
print("\n--- Количество посетителей, сделавших покупку, в каждой группе ---")
print(purchasers_per_group)



--- Общее количество посетителей в каждой группе ---
group
A    4952
B    5011
Name: user_id, dtype: int64

--- Количество посетителей, сделавших покупку, в каждой группе ---
group
A    1411
B    1519
Name: user_id, dtype: int64


In [21]:
# Расчет и вывод конверсии для каждой группы  
for group_name in total_visitors_per_group.index:
    total = total_visitors_per_group[group_name]
    purchasers = purchasers_per_group[group_name] 

    conversion_rate = (purchasers / total) * 100
    print(f"Группа '{group_name}':")
    print(f"  Всего посетителей: {total}")
    print(f"  Покупателей: {purchasers}")
    print(f"  Конверсия: {conversion_rate:.2f}%")

Группа 'A':
  Всего посетителей: 4952
  Покупателей: 1411
  Конверсия: 28.49%
Группа 'B':
  Всего посетителей: 5011
  Покупателей: 1519
  Конверсия: 30.31%


- сделайте предварительный общий вывод об изменении пользовательской активности в тестовой группе по сравнению с контрольной.

Согласно полученным результатам, наблюдается увеличение конверсии с тестовой группе по сравнению с контрольной (с 28.5 до 30.3%). Необходимо оценить, является ли такое изменение статистически значимым.

## 4. Проведите оценку результатов A/B-тестирования:

- Проверьте изменение конверсии подходящим статистическим тестом, учитывая все этапы проверки гипотез.

Для рассчета статистической значимости используем Z-тест пропорций. 

Предположим, что нулевая гипотеза формулируется так: нет значимых различий в конверсии пользователей группы А и В.

В таком случае альтернативная гипотеза: конверсия в тестовой группе В больше конверсии в контрольной группе А.

In [22]:
# Размеры выборок А и В
n_a = total_visitors_per_group['A']
n_b = total_visitors_per_group['B']

# Количество успехов (пользователей с покупками) в группах А и В
m_a = purchasers_per_group['A']
m_b = purchasers_per_group['B']

p_a = m_a / n_a  # рассчитываем доли успехов для группы A
p_b = m_b / n_b  # рассчитываем доли успехов для группы B

print(f'n_a={n_a}, n_b={n_b}')
print(f'm_a={m_a}, m_b={m_b}')
print(f'p_a={p_a:.4f}, p_b={p_b:.4f}')

# Проверка предпосылки о достаточном количестве данных
# Условие: количество успехов и неудач в каждой группе должно быть > 10
if (m_a > 10) and ((n_a - m_a) > 10) and (m_b > 10) and ((n_b - m_b) > 10):
    print('Предпосылка о достаточном количестве данных выполняется!')
else:
    print('Предпосылка о достаточном количестве данных НЕ выполняется!')

n_a=4952, n_b=5011
m_a=1411, m_b=1519
p_a=0.2849, p_b=0.3031
Предпосылка о достаточном количестве данных выполняется!


In [24]:
alpha = 0.05 ## на каком уровне значимости проверяем гипотезу о равенстве вероятностей

stat_ztest, p_value_ztest = proportions_ztest(
    [m_a, m_b],
    [n_a, n_b],
    alternative='smaller' # так как H_1: p_a < p_b
)
p_value_ztest

if p_value_ztest > alpha:
    print(f'pvalue={p_value_ztest} > {alpha}')
    print('Нулевая гипотеза находит подтверждение!')
    print('Нет значимых различий в конверсии пользователей группы А и В.')

else:
    print(f'pvalue={p_value_ztest} < {alpha}')
    print('Нулевая гипотеза не находит подтверждения!')
    print('Конверсия в тестовой группе В больше конверсии в контрольной группе А и это различие стастически значимо.')

pvalue=0.023117327881967534 < 0.05
Нулевая гипотеза не находит подтверждения!
Конверсия в тестовой группе В больше конверсии в контрольной группе А и это различие стастически значимо.


- Опишите выводы по проведённой оценке результатов A/B-тестирования. Что можно сказать про результаты A/B-тестирования? Был ли достигнут ожидаемый эффект в изменении конверсии?

По результатам оценки A/B-тестирования можно сделать следующий вывод: 
- Исследуемая гипотеза подтвердилась, был достигнут ожидаемый эффект.
- Упрощение интерфейса привело к тому, что в течение семи дней после регистрации конверсия зарегистрированных пользователей в покупателей увеличилась, и это изменение статистически значимо.